<a href="https://colab.research.google.com/github/amyyq2/CourseProject/blob/main/comments_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [102]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd   
from selenium import webdriver

data=[]
# driver = webdriver.Chrome('/path/to/chromedriver') 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

with Chrome('chromedriver',chrome_options=chrome_options) as driver:
    wait = WebDriverWait(driver,15)
    # driver.get("https://www.youtube.com/watch?v=0EVVKs6DQLo") # kiss me more
    driver.get("https://www.youtube.com/watch?v=YbJOTdZBX1g") # baby shark

    for item in range(40): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        # time.sleep(15)

    comments = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content")))
    for comment in comments:
        data.append(comment.text)

df = pd.DataFrame(data, columns=['comment'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [103]:
%load_ext google.colab.data_table
df.iloc[2: , :]

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,comment
2,Let’s all take a moment of silence for the 19M...
3,Its nice that this video has no dislikes!\nit ...
4,those 19M dislikes will never be forgotten. R....
5,3M likes with no dislikes! Woah this video mus...
6,I cant believe 3 million people liked this and...
...,...
137,removing dislikes just shows how much is Youtu...
138,We are all talking bout the 19 million dislike...
139,rewatching this made me realize how out of tou...
140,I literally have completely no idea who 90% of...


# Data Cleaning
- Filter out non-english comments
- Filter out EXACT duplicate comments


In [104]:
"""
Remove first two rows
"""
df = df.drop([0, 1])

In [105]:
'''
Removing all non-english comments
'''
!pip install langdetect
from langdetect import detect

In [106]:

def detect_lang(text):
    return detect(text)

df.drop(df[df['comment'].map(detect_lang) != "en"].index, inplace = True)

In [107]:
%load_ext google.colab.data_table
df.iloc[: , :]

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,comment
2,Let’s all take a moment of silence for the 19M...
3,Its nice that this video has no dislikes!\nit ...
4,those 19M dislikes will never be forgotten. R....
5,3M likes with no dislikes! Woah this video mus...
6,I cant believe 3 million people liked this and...
...,...
137,removing dislikes just shows how much is Youtu...
138,We are all talking bout the 19 million dislike...
139,rewatching this made me realize how out of tou...
140,I literally have completely no idea who 90% of...


In [108]:
'''
Removing duplicate comments
'''

df.drop_duplicates()

,comment
2,Let’s all take a moment of silence for the 19M...
3,Its nice that this video has no dislikes!\nit ...
4,those 19M dislikes will never be forgotten. R....
5,3M likes with no dislikes! Woah this video mus...
6,I cant believe 3 million people liked this and...
...,...
137,removing dislikes just shows how much is Youtu...
138,We are all talking bout the 19 million dislike...
139,rewatching this made me realize how out of tou...
140,I literally have completely no idea who 90% of...


# Sentiment Analysis

**Using the Vader Sentiment Analysis package, here is how we will using the output to classify the output sentiment ([from the docs](https://github.com/cjhutto/vaderSentiment)):**

The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

positive sentiment: compound score >= 0.05
neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
negative sentiment: compound score <= -0.05

In [109]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.0 requires urllib3[secure]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [110]:
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    # print("Overall sentiment dictionary is : ", sentiment_dict)
    # print("The compound score is: ", sentiment_dict["compound"])
    compound = sentiment_dict["compound"]
    # print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    # print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    # print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
 
    # print("Sentence Overall Rated As", end = " ")

    return compound
 

 

def avg_comment_sentiment(comment_df):

    # convert df to list
    comment_list = comment_df.values.tolist()

    comment_sentiments = [sentiment_scores(comment) for comment in comment_list]

    # print("the comment sentiments are: ", comment_sentiments)

    sentiment_score_total = 0
    count = 0
    for sentiment in comment_sentiments:
        count += 1
        sentiment_score_total = sentiment_score_total + sentiment


    print("sentiment score total is: ", sentiment_score_total)
    print("# comments is: ", count)
    overall_sentiment = sentiment_score_total/count

    print("overall sentiment is: ", overall_sentiment)


    # decide sentiment as positive, negative and neutral
    if overall_sentiment >= 0.05 :
        print("Positive")
 
    elif overall_sentiment <= - 0.05 :
        print("Negative")
 
    else :
        print("Neutral")


In [111]:
test1 = "I hate you"
test2 = "they ATE they ATE they A T E"

sentiment_scores(test1)
sentiment_scores(test2)

0.0

In [112]:
avg_comment_sentiment(df)

sentiment score total is:  -2.9960000000000004
# comments is:  133
overall sentiment is:  -0.022526315789473686
Neutral


# Parallelization/Multi-Threading